# NLP Exercise 5: BERTs with Masked Language Model and Named Entities Recognition 
---

## Masked Language Model

### Define libraries and dataset

In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM
from torch.optim import AdamW
from transformers import get_scheduler
from transformers import default_data_collator
from tqdm.auto import tqdm
import math

In [8]:
# Load the dataset
data = load_dataset('imdb')
print(data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [11]:
print(data["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

### Define Tokenizer

In [18]:
# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize function to tokenize the whole dataset
def tokenize(data):
    tokenized_data = tokenizer(data["text"])
    return tokenized_data

# Map function 
tokenized_data = data.map(tokenize, batched=True, remove_columns=["text", "label"])

Map: 100%|██████████| 50000/50000 [00:17<00:00, 2921.19 examples/s]


In [19]:
print(tokenized_data)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 50000
    })
})


### Preprocessing for training loop

In [21]:
def group_texts(data):
    chunk_size = 128
    # concatenate texts
    concatenated_sequences = {k: sum(data[k], []) for k in data.keys()}
    # calculate the total number of tokens after concatenation
    total_concat_length = len(concatenated_sequences[list(data.keys())[0]])
    # drop the last chunk if is smaller than the chunk size
    total_length = (total_concat_length // chunk_size) * chunk_size

    # split the concatenated sentences into chunks using the total length
    result = {k: [t[i: i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_sequences.items()}

    '''we create a new labels column which is a copy of the input_ids of the processed text data, 
    we need to predict randomly masked tokens in the input batch and the labels column serve as 
    ground truth for our masked language model to learn from. '''
    
    result["labels"] = result["input_ids"].copy()

    return result

processed_dataset = tokenized_data.map(group_texts, batched = True)


Map: 100%|██████████| 50000/50000 [01:56<00:00, 430.93 examples/s]


Example for MLM
```
input_ids  = [101, 2009, 2001, 103, 1012]   # 'It was [MASK].'
labels     = [101, 2009, 2001, 2204, 1012]  # 'It was good.'

```

In [27]:
# Resize the dataset to 10000 samples(optional, just for low processing units)
train_size = 10000
test_size = int(0.1 * train_size)

downsampled_dataset = processed_dataset["train"].train_test_split(train_size=train_size, test_size=test_size, seed=42)

Apply random masking on the whole test data, then uses the default data collector to handle the test dataset in batches.

In [28]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15)

# we shall insert mask randomly in the sentence
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

We drop the unmasked columns in the test dataset and replace them with the masked ones

In [29]:
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2475.63 examples/s]


### Define DataLoader and relevant params

In [32]:
batch_size = 64

# load the train dataset for traing
train_dataloader = DataLoader(downsampled_dataset["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator,)
# load the test dataset for evaluation
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=default_data_collator)

# initialize pretrained bert model
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')

# set the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

### Traning loop

In [ ]:
# Define the number of epochs
epochs = 10

# Define the learning rate scheduler
num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training and evaluation loop
progress_bar = tqdm(range(num_training_steps))

for epoch in range(epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    eval_loss = 0
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        eval_loss += outputs.loss.item()

    eval_loss = eval_loss / len(eval_dataloader)
    perplexity = math.exp(eval_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {loss.item()}, Evaluation Loss: {eval_loss}, Perplexity: {perplexity}")